# Wrestling with Structured Output
```{epigraph}
In limits, there is freedom. Creativity thrives within structure.

-- Julia B. Cameron
```
```{contents}
```

## The Structured Output Challenges

Large language models (LLMs) excel at generating human-like text, but they often struggle to produce output in a structured format consistently. This poses a significant challenge when we need LLMs to generate data that can be easily processed by other systems, such as databases, APIs, or other software applications.  

Sometimes, even with a well-crafted prompt, an LLM might produce an unstructured response when a structured one is expected. This can be particularly challenging when integrating LLMs into systems that require specific data formats.

Throughout this notebook, we will consider as input a segment of a sample SEC filing of Apple Inc.

In [1]:
MAX_LENGTH = 10000 # We limit the input length to avoid token issues
with open('../data/apple.txt', 'r') as file:
    sec_filing = file.read()
sec_filing = sec_filing[:MAX_LENGTH] 

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

True

In [2]:
from openai import OpenAI

In [35]:
client = OpenAI()
# Define the prompt expecting a structured JSON response
prompt = f"""
Generate a two-person discussion about the key financial data from the following text in JSON format.
TEXT: {sec_filing}
"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)

In [26]:
response_content = response.choices[0].message.content
print(response_content)

Person 1: Wow, Apple Inc. seems to have a lot of different products and services they offer. It's interesting to see the breakdown of their revenue streams in their Form 10-K.

Person 2: Absolutely, they have a diverse portfolio with iPhones, Macs, iPads, wearables, and even services. It's impressive to see how they have capitalized on different technology trends.

Person 1: I noticed that they have a large market value of over $2.6 trillion as of March 29, 2024. That's a huge amount, and it shows the confidence investors have in the company.

Person 2: Definitely, that's a significant figure. It's also good to see that they are complying with all the required SEC regulations and filing their reports in a timely manner.

Person 1: Yes, it's crucial for investors to have access to accurate and up-to-date financial information. It helps in making informed decisions about their investments in the company.

Person 2: Absolutely, transparency and compliance with regulations are key in the f

In [3]:
import json

def is_json(myjson):
  try:
    json.loads(myjson)
  except ValueError as e:
    return False
  return True

In [4]:
is_json(response_content)

False


In this example, despite the prompt clearly asking for a JSON object, the LLM generates a natural language sentence instead. This highlights the inconsistency and unpredictability of LLMs when it comes to producing structured output.

## Problem Statement

Obtaining structured output from LLMs presents several significant challenges:

* **Inconsistency**: LLMs often produce unpredictable results, sometimes generating well-structured output and other times deviating from the expected format.

* **Lack of Type Safety**: LLMs do not inherently understand data types, which can lead to errors when their output is integrated with systems requiring specific data formats.

* **Prompt Engineering Complexity**: Crafting prompts that effectively guide LLMs to produce the correct structured output is complex and requires extensive experimentation.

## Solutions

Several strategies and tools can be employed to address the challenges of structured output from LLMs.

### Strategies

* **Schema Guidance**: Providing the LLM with a clear schema or blueprint of the desired output structure helps to constrain its generation and improve consistency. This can be achieved by using tools like Pydantic to define the expected data structure and then using that definition to guide the LLM's output. 

* **Output Parsing**: When LLMs don't natively support structured output, parsing their text output using techniques like regular expressions or dedicated parsing libraries can extract the desired information. For example, you can use regular expressions to extract specific patterns from the LLM's output, or you can use libraries like Pydantic to parse the output into structured data objects.

* **Type Enforcement**: Using tools that enforce data types, such as Pydantic in Python, can ensure that the LLM output adheres to the required data formats. This can help to prevent errors when integrating the LLM's output with other systems.

### Techniques and Tools

#### One-Shot Prompts

In one-shot prompting, you provide a single example of the desired output format within the prompt.

In [31]:
prompt = f"""
Generate a two-person discussion about the key financial data from the following text in JSON format.

<JSON_FORMAT>
{
   "Person1": {
     "name": "Alice",
     "statement": "The revenue for Q1 has increased by 20% compared to last year."
   },
   "Person2": {
     "name": "Bob",
     "statement": "That's great news! What about the net profit margin?"
   }
}
</JSON_FORMAT>

TEXT: {sec_filing}
"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)

In [32]:
response_content = response.choices[0].message.content
print(response_content)

{
   "Person1": {
     "name": "Alice",
     "statement": "The revenue for Q1 has increased by 20% compared to last year."
   },
   "Person2": {
     "name": "Bob",
     "statement": "That's great news! What about the net profit margin?"
   }
}


In [33]:
is_json(response_content)


True

#### Structured Output with Provider-Specific APIs

One-shot prompting is a simple technique that can lead to material improvements in structured output, though may not be sufficient for complex (e.g. nested) structures and / or when the model's output needs to be restricted to a specific set of options or types.

Provider-specific APIs can offer ways to handle those challenges. We will explore two approaches here using OpenAI's API:

* **JSON Mode**: Most LLM APIs today offer features specifically designed for generating JSON output.
* **Structured Outputs**: Some LLM APIs offer features specifically designed for generating structured outputs with type safety.

#### JSON Mode

JSON mode is a feature provided by most LLM API providers, such as OpenAI, that allows the model to generate output in JSON format. This is particularly useful when you need structured data as a result, such as when parsing the output programmatically or integrating it with other systems that require JSON input. As depicted in {numref}`json-mode`, JSON mode is implemented by instructing theLLM model to use JSON as response format and optionally defining a target schema.


```{figure} ../_static/structured_output/json.png
---
name: json-mode
alt: JSON Mode
scale: 50%
align: center
---
Conceptual overview of JSON mode.
```

When using JSON mode with OpenAI's API, it is recommended to instruct the model to produce JSON via some message in the conversation, for example via your system message. If you don't include an explicit instruction to generate JSON, the model may generate an unending stream of whitespace and the request may run continually until it reaches the token limit. To help ensure you don't forget, the API will throw an error if the string "JSON" does not appear somewhere in the context.




In [48]:
prompt = f"""
Generate a two-person discussion about the key financial data from the following text in JSON format.
TEXT: {sec_filing}
"""
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
response_format = { "type": "json_object" }
)

In [49]:
response_content = response.choices[0].message.content
print(response_content)

{
  "person1": "I see that Apple Inc. reported a total market value of approximately $2,628,553,000,000 held by non-affiliates as of March 29, 2024. That's a significant amount!",
  "person2": "Yes, it definitely shows the scale and value of the company in the market. It's impressive to see the sheer size of the market value.",
  "person1": "Also, they mentioned having 15,115,823,000 shares of common stock issued and outstanding as of October 18, 2024. That's a large number of shares circulating in the market.",
  "person2": "Absolutely, the number of shares outstanding plays a crucial role in determining the company's market capitalization and investor interest."
}


In [50]:
is_json(response_content)

True

This example solution is specific to OpenAI's API. Other LLM providers offer similar functionality, for example:

* Google's Vertex AI offers a `parse` method for structured outputs.
* Anthropic offers a `structured` method for structured outputs.

JSON mode will not guarantee the output matches any specific schema, only that it is valid and parses without errors. For that purpose, we can leverage a new feature recently released by OpenAI called "Structured Outputs" to ensure the output data matches a target schema with type safety.


**Structured Output Mode**

Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema, so you don't need to worry about the model omitting a required key, or hallucinating an invalid enum value.

Some benefits of Structured Outputs include:
- **Reliable type-safety**: No need to validate or retry incorrectly formatted responses.
- **Explicit refusals**: Safety-based model refusals are now programmatically detectable.
- **Simpler prompting**: No need for strongly worded prompts to achieve consistent formatting.


Here's a Python example demonstrating how to use the OpenAI API to generate a structured output. In this example, we aim at extracting structured data from our sample SEC filing, in particular: (i) entities and (ii) places mentioned in the input doc. This example uses the `response_format` parameter within the OpenAI API call. This functionality is supported by GPT-4o models, specifically `gpt-4o-mini-2024-07-18`, `gpt-4o-2024-08-06`, and later versions.

In [4]:
from pydantic import BaseModel
from openai import OpenAI

class SECExtraction(BaseModel):
    mentioned_entities: list[str]
    mentioned_places: list[str]

def extract_from_sec_filing(sec_filing_text: str, prompt: str) -> SECExtraction:
    """
    Extracts structured data from an input SEC filing text.
    """
    client = OpenAI()
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {"role": "user", "content": sec_filing_text}
        ],
        response_format=SECExtraction
    )
    return completion.choices[0].message.parsed

**Explanation:**

*   **Data Structures:** The code defines one Pydantic model, `SECExtraction`, to represent the structured output of our parser. This model provide type hints and structure for the response.
*   **API Interaction:** The `extract_from_sec_filing` function uses the OpenAI client to send a chat completion request to the `gpt-4o-mini-2024-07-18` model. The prompt instructs the model to extract our target attributes from input text. The `response_format` is set to `SECExtraction`, ensuring the response conforms to the specified Pydantic model.
*   **Output Processing:** The returned response is parsed into the `SECExtraction` model. The code then returns the parsed data.

In [69]:
prompt_extraction = "You are an expert at structured data extraction. You will be given unstructured text from a SEC filing and extracted names of mentioned entities and places and should convert the response into the given structure."
sec_extraction = extract_from_sec_filing(sec_filing, prompt_extraction)

In [70]:
print("Extracted entities:", sec_extraction.mentioned_entities)
print("Extracted places:", sec_extraction.mentioned_places)


Extracted entities: ['Apple Inc.', 'The Nasdaq Stock Market LLC']
Extracted places: ['Washington, D.C.', 'California', 'Cupertino, California']


We observe that the model was able to extract the entities and places from the input text, and return them in the specified format.

**Benefits**

*   **Structured Output:** The use of Pydantic models and the `response_format` parameter enforces the structure of the model's output, making it more reliable and easier to process.

*   **Schema Adherence:**  Structured Outputs in OpenAI API guarantee that the response adheres to the provided schema.

This structured approach improves the reliability and usability of your application by ensuring consistent, predictable output from the OpenAI API.

This example solution is specific to OpenAI's API. That begs the question: How can we solve this problem generally for widely available LLM providers? In the next sections, we will explore how `LangChain` and `Outlines` may serve as general purpose tools that can help us do just that.


### LangChain

LangChain is a framework designed to simplify the development of LLM applications. It provider an abstraction layer over many LLM providers, including OpenAI, that offers several tools for parsing structured output.

In particular, LangChain offers the `with_structured_output` method, which can be used with LLMs that support structured output APIs, allowing you to enforce a schema directly within the prompt.

> `with_structured_output` takes a schema as input which specifies the names, types, and descriptions of the desired output attributes. The method returns a model-like Runnable, except that instead of outputting strings or messages it outputs objects corresponding to the given schema. The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class. If TypedDict or JSON Schema are used then a dictionary will be returned by the Runnable, and if a Pydantic class is used then a Pydantic object will be returned.


```bash
pip install -qU langchain-openai
```

In [110]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
def extract_from_sec_filing_langchain(sec_filing_text: str, prompt: str) -> SECExtraction:
    """
    Extracts structured data from an input SEC filing text using LangChain.
    """
    llm = ChatOpenAI(model="gpt-4o-mini")

    structured_llm = llm.with_structured_output(SECExtraction)

    prompt_template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("human", "{sec_filing_text}"),
        ]
    )

    llm_chain = prompt_template | structured_llm
    
    return llm_chain.invoke(sec_filing_text)

In [111]:

prompt_extraction = "You are an expert at structured data extraction. You will be given unstructured text from a SEC filing and extracted names of mentioned entities and places and should convert the response into the given structure."
sec_extraction_langchain = extract_from_sec_filing_langchain(sec_filing, prompt_extraction)


In [112]:
print("Extracted entities:", sec_extraction_langchain.mentioned_entities)
print("Extracted places:", sec_extraction_langchain.mentioned_places)

Extracted entities: ['Apple Inc.']
Extracted places: ['California', 'Cupertino']


We observe that the model was able to extract the entities and places from the input text, and return them in the specified format. A full list of models that support `.with_structured_output()` can be found [here](https://python.langchain.com/docs/integrations/chat/#featured-providers).

### Outlines

Outlines {cite}`outlines2024` is a library specifically focused on structured text generation from LLMs. Under the hood, Outlines works by adjusting the probability distribution of the model's output logits - the raw scores from the final layer of the neural network that are normally converted into text tokens. By introducing carefully crafted logit biases, Outlines can guide the model to prefer certain tokens over others, effectively constraining its outputs to a predefined set of valid options. This provides fine-grained control over the model's generation process. In that way, Outlines provides several powerful features:

* **Multiple Choice Generation**: Restrict the LLM output to a predefined set of options.
* **Regex-based structured generation**: Guide the generation process using regular expressions.
* **Pydantic model**: Ensure the LLM output follows a Pydantic model.
* **JSON Schema**: Ensure the LLM output follows a JSON Schema.

Outlines can support major proprietary LLM APIs (e.g. OpenAI's via vLLM). However, one of its key advantages is the ability to ensure structured output for Open Source models, which often lack such guarantees by default.

```bash
pip install outlines
pip install transformers
```

In this example, we will use a Qwen2.5-0.5B model, a lightweight open source model from Alibaba Cloud known for its strong performance despite its small size. The model excels at instruction following and structured generation tasks while being efficient enough to run locally via Hugging Face's `transformers` library.

In [10]:
import outlines

model = outlines.models.transformers("Qwen/Qwen2.5-0.5B-Instruct")

#### A Simple Example: Multiple Choice Generation

In [11]:
TOP = 100
prompt = f"""You are a sentiment-labelling assistant specialized in Financial Statements.
Is the following document positive or negative?

Document: {sec_filing[:TOP]}
"""

generator = outlines.generate.choice(model, ["Positive", "Negative"])
answer = generator(prompt)
print(answer)

Negative


In this simple example, we use Outlines' `choice` method to constrain the model output to a predefined set of options ("Positive" or "Negative"). This ensures the model can only return one of these values, avoiding any unexpected or malformed responses.


#### Pydantic model

Outlines allows to guide the generation process so the output is guaranteed to follow a JSON schema or Pydantic model. Now we will go back to our example of extracting entities and places from a SEC filing. In order to do so, we simply need to pass our Pydantic model to the `json` method in Outlines' `generate` module.

In [12]:
prompt = f"You are an expert at structured data extraction. You will be given unstructured text from a SEC filing and extracted names of mentioned entities and places and should convert the response into the given structure. Document: {sec_filing[:TOP]} "
generator = outlines.generate.json(model, SECExtraction)
sec_extraction_outlines = generator(prompt)

In [13]:
print("Extracted entities:", sec_extraction_outlines.mentioned_entities)
print("Extracted places:", sec_extraction_outlines.mentioned_places)


Extracted entities: ['Zsp', 'ZiCorp']
Extracted places: ['California']


We observe that the model was able to extract the entities and places from the input text, and return them in the specified format. However, it is interesting to see that the model hallucinates a few entities, a phenomenon that is common for smaller Open Source models that were not fine-tuned on the task of entity extraction.

## Discussion

### Comparing Solutions

* **Simplicity vs. Control**: One-shot prompts are simple but offer limited control.  `LangChain`, and Outlines provide greater control but might have a steeper learning curve though quite manageable.

* **Native LLM Support**:  `with_structured_output` in LangChain relies on the underlying LLM having built-in support for structured output APIs, i.e. LangChain is a wrapper around the underlying LLM's structured output API. Outlines, on the other hand, is more broadly applicable enabling a wider range of Open Source models.

* **Flexibility**:  Outlines and LangChain's  `StructuredOutputParser`  offer the most flexibility for defining custom output structures.

### Best Practices


* **Clear Schema Definition**: Define the desired output structure clearly. This can be done in several ways including schemas, types, or Pydantic models as appropriate. This ensures the LLM knows exactly what format is expected.

* **Descriptive Naming**: Use meaningful names for fields and elements in your schema. This makes the output more understandable and easier to work with.

* **Detailed Prompting**: Guide the LLM with well-crafted prompts that include examples and clear instructions.  A well-structured prompt improves the chances of getting the desired output.

* **Integration**: If you are connecting the model to tools, functions, data, etc. in your system, then you are highly encouraged to use a typed structured output (e.g. Pydantic models) to ensure the model's output can be processed correctly by downstream systems.



### Ongoing Debate on LLMs Structured Output

The use of structured output, like JSON or XML, for Large Language Models (LLMs) is a developing area. While structured output offers clear benefits in parsing, robustness, and integration, there is growing debate on whether it also potentially comes at the cost of reasoning abilities. 

Recent research "Let Me Speak Freely? A Study on the Impact of Format Restrictions on Performance of Large Language Models" {cite}`tam2024letspeakfreelystudy` suggests that imposing format restrictions on LLMs might impact their performance, particularly in reasoning-intensive tasks. Further evidence {cite}`aider2024codejson` suggests LLMs may produce lower quality code if they’re asked to return it as part of a structured JSON response, in particular:

* **Potential performance degradation:**  Enforcing structured output, especially through constrained decoding methods like JSON-mode, can negatively impact an LLM's reasoning abilities. This is particularly evident in tasks that require multi-step reasoning or complex thought processes.

* **Overly restrictive schemas:** Imposing strict schemas can limit the expressiveness of LLM outputs and may hinder their ability to generate creative or nuanced responses.  In certain cases, the strictness of the schema might outweigh the benefits of structured output.

* **Increased complexity in prompt engineering:** Crafting prompts that effectively guide LLMs to generate structured outputs while maintaining performance can be challenging. It often requires careful consideration of the schema, the task instructions, and the desired level of detail in the response.

On the other hand, those findings are not without criticism. The .txt team challenges the work of {cite}`tam2024letspeakfreelystudy`. The rebuttal argues that **structured generation, when done correctly, actually *improves* performance**.


```{figure} ../_static/structured_output/rebuttal.png
---
name: structured_vs_unstructured
alt: Structured vs Unstructured Results by .txt team
scale: 60%
align: center
---
Structured vs Unstructured Results by .txt team.
```

The .txt team presents compelling evidence through their reproduction of the paper's experiments. While their unstructured results align with the original paper's findings, their structured results paint a dramatically different picture - demonstrating that structured generation actually improves performance (see {numref}`structured_vs_unstructured`). The team has made their experimental notebooks publicly available on GitHub for independent verification {cite}`dottxt2024demos`.


.txt team identifies several flaws in the methodology of "Let Me Speak Freely?" that they believe led to inaccurate conclusions:

*   The paper finds that structured output improves performance on classification tasks but doesn't reconcile this finding with its overall negative conclusion about structured output. 
*   The prompts used for unstructured generation were different from those used for structured generation, making the comparison uneven. 
*   The prompts used for structured generation, particularly in JSON-mode, didn't provide the LLM with sufficient information to properly complete the task. 
*   The paper conflates "structured generation" with "JSON-mode", when they are not the same thing. 

It is important to note that while .txt provides a compelling and verifiable argument in favor of (proper) structured output generation in LLMs further research and exploration are needed to comprehensively understand the nuances and trade-offs involved in using structured output for various LLM tasks and applications.

In summary, the debate surrounding structured output highlights the ongoing challenges in balancing LLM capabilities with real-world application requirements. While structured outputs offer clear benefits in parsing, robustness, and integration, their potential impact on performance, particularly in reasoning tasks is a topic of ongoing debate. 

The ideal approach likely involves a nuanced strategy that considers the specific task, the desired level of structure, and the available LLM capabilities. Further research and development efforts are needed to mitigate potential drawbacks and unlock the full potential of LLMs for a wider range of applications. 


## Conclusion

Extracting structured output from LLMs is crucial for integrating them into real-world applications. By understanding the challenges and employing appropriate strategies and tools, developers can improve the reliability and usability of LLM-powered systems, unlocking their potential to automate complex tasks and generate valuable insights. 

## Acknowledgements

We would like to thank Cameron Pfiffer from the .txt team for his insightful review and feedback.


## References
```{bibliography}
:filter: docname in docnames
```